<a href="https://colab.research.google.com/github/bkokalari/Doctoranytime-Automatic-Review-Acceptance-Model/blob/main/DataPreprocessing_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install Unidecode

     |████████████████████████████████| 245kB 3.4MB/s 


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.core.display import HTML 
import spacy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import nltk
import itertools   
import string
import imblearn
from collections import Counter
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from scipy import stats
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
revs = pd.read_csv("/content/drive/My Drive/Colab Notebooks/reviews_moderation_v2/data/English/augmented_data_english.csv")
revs

,TranslatedReview,Approved
0,"Reassuring, experienced and effective",1
1,A wonderful person makes you trust her and be ...,1
2,"The doctor is excellent, familiar with the sub...",1
3,It gave me confidence and I said things I have...,1
4,Very good doctor I highly recommend him. Absol...,1
...,...,...
131574,Big guy the guy. He created a problem in my st...,0
131575,Καθίκι !! I have nothing else to say!,0
131576,After the treatment I started to feel dizzy !!...,0
131577,I got a checkered rash all over my body with t...,0


In [ ]:
import re


def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

In [ ]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [ ]:
punctuations = string.punctuation

def remove_punctuation(text):
  punctuated_string = text
  unpunctuated_string = punctuated_string.translate(str.maketrans('', '', string.punctuation))
  return unpunctuated_string

In [ ]:
revs.TranslatedReview

0                     Reassuring, experienced and effective
1         A wonderful person makes you trust her and be ...
2         The doctor is excellent, familiar with the sub...
3         It gave me confidence and I said things I have...
4         Very good doctor I highly recommend him. Absol...
                                ...                        
131574    Big guy the guy. He created a problem in my st...
131575                Καθίκι !! I have nothing else to say!
131576    After the treatment I started to feel dizzy !!...
131577    I got a checkered rash all over my body with t...
131578    The guy was the biggest sexist I have ever met...
Name: TranslatedReview, Length: 131579, dtype: object

In [ ]:
revs["TranslatedReview"] = revs.TranslatedReview.map(lambda x: remove_URL(str(x)))
revs["TranslatedReview"] = revs.TranslatedReview.map(lambda x: remove_html(x))
revs["TranslatedReview"] = revs.TranslatedReview.map(lambda x: remove_emoji(x))
revs["TranslatedReview"] = revs.TranslatedReview.map(lambda x: remove_punctuation(x))

In [ ]:
revs[330:360]

,TranslatedReview,Approved
330,Excellent doctor approachable man,1
331,Analytical and explanatory right professional ...,1
332,Very detailed examination if k was only for a ...,1
333,First of all the place is wonderful The staff ...,1
334,Although I went for a simple swimming pool cer...,1
335,Perfect and a lot of service at the time of th...,1
336,Immediate and practical Gentle and intimate,1
337,is a good doctor and effective,1
338,At the moment it was just the first visit I am...,1
339,On time and he made me feel happy and comforta...,1


In [ ]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))


def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [ ]:
stop.remove('not')
stop.remove('couldn')
stop.remove('didn')
stop.remove('doesn')

In [ ]:
revs["TranslatedReview"] = revs["TranslatedReview"].map(remove_stopwords)

In [ ]:
revs[330:360]

,TranslatedReview,Approved
330,excellent doctor approachable man,1
331,analytical explanatory right professional pers...,1
332,detailed examination k swimming pool certifica...,1
333,first place wonderful staff kind helpful docto...,1
334,although went simple swimming pool certificate...,1
335,perfect lot service time appointment,1
336,immediate practical gentle intimate,1
337,good doctor effective,1
338,moment first visit happy,1
339,time made feel happy comfortable without losin...,1


In [ ]:
#revs.to_csv('/content/drive/My Drive/Colab Notebooks/reviews_moderation_v2/data/Greek/preprocessed_Greek.csv', header=True, index=False)